# Hotel Booking Cancellation Prediction

!["Image of hotel resort in the mountains with a swimming pool in the foreground"](images/hotel-pool.jpg)

## Overview

This project aims to develop a predictive model that can forecast hotel booking cancellations based on historical data. The model will be trained [this dataset](https://www.kaggle.com/datasets/ahsan81/hotel-reservations-classification-dataset) obtained from Kaggle, which contains a large number of hotel reservations made by customers. The primary objective of this project is to build a machine learning model that can accurately predict the likelihood of a hotel booking being cancelled. This information can be valuable for hotels as it allows them to make informed decisions about their inventory management, staff scheduling, and revenue optimization strategies.

The project will involve several steps, including data cleaning, exploratory data analysis, feature engineering, model selection, and evaluation. The performance of the model will be measured using metrics such as accuracy, precision, recall, and F1-score.

Throughout this project, we will also explore the relationships between different variables and their impact on booking cancellations. This will help us to gain insights into the factors that contribute to cancellations and to identify potential areas for improvement. Overall, this project has the potential to provide valuable insights and practical applications for the hospitality industry. By developing a predictive model that can accurately forecast booking cancellations, hotels can better manage their resources, improve customer satisfaction, and increase revenue.

## Business Understanding

The hospitality industry is a highly competitive market, with hotels constantly striving to optimize their revenue and improve customer satisfaction. One of the major challenges faced by hotels is managing cancellations, which can result in lost revenue and reduced occupancy rates.

According to a [recent study](https://www.hotelmanagement.net/tech/study-cancelation-rate-at-40-as-otas-push-free-change-policy), average hotel cancellations have risen to at least 40% of overall bookings. This highlights the need for a predictive model that can accurately forecast booking cancellations. By building a predictive model that can accurately forecast booking cancellations, hotels can take proactive measures to minimize the impact of cancellations on their revenue. For example, they can offer discounts or special promotions to customers who are likely to cancel their bookings, or they can allocate resources more efficiently by adjusting their staff scheduling and inventory management strategies based on the predicted cancellation rates.

Overall, the business value of this project lies in its ability to help hotels improve their revenue optimization strategies, increase customer satisfaction, and reduce the financial impact of booking cancellations. By developing a predictive model that can accurately forecast booking cancellations, hotels can gain a competitive edge in the highly competitive hospitality industry.

## Data Understanding

To achieve this objective, I have utilized the Hotel Reservations [Dataset](https://www.kaggle.com/datasets/ahsan81/hotel-reservations-classification-dataset) obtained from Kaggle. The dataset contains information about hotel bookings made by customers, including various features such as the number of adults and children, the type of meal plan, the requirement for a car parking space, the type of room reserved, the lead time, the arrival date, the market segment type, and whether the booking was cancelled or not.

The dataset consists of 32,000 rows and 19 columns, with each row representing a unique booking. The target variable is the `booking_status` column, which indicates whether the booking was cancelled (1) or not (0).

The dataset includes the following features:
- Booking_ID: unique identifier of each booking
- no_of_adults: Number of adults
- no_of_children: Number of Children
- no_of_weekend_nights: Number of weekend nights (Saturday or Sunday) the guest stayed or booked to stay at the hotel
- no_of_week_nights: Number of week nights (Monday to Friday) the guest stayed or booked to stay at the hotel
- type_of_meal_plan: Type of meal plan booked by the customer:
- required_car_parking_space: Does the customer require a car parking space? (0 - No, 1- Yes)
- room_type_reserved: Type of room reserved by the customer. The values are ciphered (encoded) by INN Hotels.
- lead_time: Number of days between the date of booking and the arrival date
- arrival_year: Year of arrival date
- arrival_month: Month of arrival date
- arrival_date: Date of the month
- market_segment_type: Market segment designation.
- repeated_guest: Is the customer a repeated guest? (0 - No, 1- Yes)
- no_of_previous_cancellations: Number of previous bookings that were canceled by the customer prior to the current booking
- no_of_previous_bookings_not_canceled: Number of previous bookings not canceled by the customer prior to the current booking
- avg_price_per_room: Average price per day of the reservation; prices of the rooms are dynamic. (in euros)
- no_of_special_requests: Total number of special requests made by the customer (e.g. high floor, view from the room, etc)
- booking_status: Flag indicating if the booking was canceled or not.

In [1]:
# Importing packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
%matplotlib inline